# Режим вопросов и ответов

Изучение метаданных паркет файла в режиме вопросов и ответов с использованием .... DUCKDB

In [1]:
import viz_tools as vt

## Метаданные

In [4]:
vt.showOverview("/home/mk/mk_win/Documents/28.Arrow/data/tpcds_10g.parquet/store_sales.parquet")

FILE: /home/mk/mk_win/Documents/28.Arrow/data/tpcds_10g.parquet/store_sales.parquet
row number       : 28800991
row groups number: 235
row_group sizes  :
┌───────────────────┬──────────────────────┐
│ rows in row_group │ number of row_groups │
│       int64       │        int64         │
├───────────────────┼──────────────────────┤
│            122880 │                  234 │
│             47071 │                    1 │
└───────────────────┴──────────────────────┘



Детальная мета-информация о колонке в пределах row group

In [5]:
vt.showColumDetals()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Какая колонка в какой row group находится по заданному смещению 

In [7]:
vt.findColumn(7166)

Offset belongs to
Row_group: 0
Column   : ss_sold_date_sk
Starts at: 7165


## Блюм фильтры

Блюм фильтры есть не у всех колонок - выберите колонку, установите значение.

Результат (придется немного подождать) - список row group, которые будут исключены из рассмотрения за счет bloom фильтров.

In [ ]:
vt.probeBloomFilters()

# Старое и отладка

In [ ]:
meta = duckdb.sql(f"SELECT * FROM parquet_metadata('{PQ_FILE}')")
sch = duckdb.sql(f"SELECT * FROM parquet_schema('{PQ_FILE}')")

Не так много, наверное, на самом деле интересно знать, а если пытаться именно смотреть метаданные - там много лишнего и глаза разбегаются.

Что интересно

статика

* количество row групп
* их размер в строках (он, насколько я понимаю, постоянен - или нет)

колонка+группа

* кодировки для колонки в группе/файле
* наличие статистики по колонке (и где именно)
* наличие блум фильтров и их размеры

* количество страниц в row группе
* ... возможно что-то еще

In [ ]:
print("FILE:",PQ_FILE)
print("row number       :",int(duckdb.sql("select sum(num_values) from meta where column_id=0").df().values.tolist()[0][0]))
print("row groups number:",duckdb.sql("select max(row_group_id)+1 from meta").df().values.tolist()[0][0])
print("rows per group   :")
duckdb.sql("select num_values, count(*) as 'group_number' from meta where column_id=0 group by 1")

In [ ]:
duckdb.sql(f"""
    select 
        path_in_schema, 
        type, 
        stats_min_value, 
        stats_max_value, 
        encodings, 
        index_page_offset, 
        bloom_filter_offset,
        bloom_filter_length
    from meta 
    where 
        row_group_id={ROW_GROUP} 
        and column_id={COLUMN}
""")

In [ ]:
ROW_GROUP = 0
COLUMN = 1

Как было бы прикольнее эту информацию получать?

In [ ]:
from IPython.display import HTML, Javascript
import ipywidgets as widgets

def doPrepLists(cols,groups):
        
    colList = widgets.Dropdown(
        options=cols,
        layout = widgets.Layout(width='200px'),
        description='Column:'
    )
    grpList = widgets.Dropdown(
        options=groups,
        layout = widgets.Layout(width='200px'),
        description='Row group:'
    )

    showBtn = widgets.Button(
        description='Show info',
        disabled=False,
        button_style='info', 
        tooltip='Prepare list for playing in player',
    )

    def doShow(b):
        
        outList.clear_output()
        with outList:
            col_type, stats_min_value, stats_max_value, encodings, index_page_offset, bloom_filter_offset,bloom_filter_length = duckdb.sql(f"""
                select 
                    type, 
                    stats_min_value, 
                    stats_max_value, 
                    encodings, 
                    index_page_offset, 
                    bloom_filter_offset,
                    bloom_filter_length
                from meta 
                where 
                    row_group_id={grpList.value} 
                    and path_in_schema='{colList.value}'
            """).df().values.tolist()[0]
            print("Column :",colList.value, "(", col_type, ")") 
            print("stats_min_value    :",stats_min_value) 
            print("stats_max_value    :",stats_max_value) 
            print("encodings          :",encodings) 
            print("index_page_offset  :",index_page_offset) 
            print("bloom_filter_offset:",bloom_filter_offset)
            print("bloom_filter_length:",bloom_filter_length)

    showBtn.on_click(doShow)

    outList = widgets.Output(layout={'border': '1px solid black'})
    display(widgets.HBox([colList,grpList,showBtn]))
    display(outList)

In [ ]:
doPrepLists(colList,range(235))

In [ ]:
colRes = duckdb.sql(f"""
    select distinct(path_in_schema)
    from meta 
    where 
        row_group_id=0
""").df().values.tolist()
colList = [ c[0] for c in colRes ]

In [ ]:
colList

Задаем файл - видим статистику (=расширенная статика)

Заполняется список колонок - из него можно выбрать колонку (начать можно просто с номеров колонок) и задать номер группы - видим статистику по колонке-группе

## Что не интересно

Много чего известно итак и из других источников

* схема
* данные как таковые
* размер
* кодек сжатия

## Пробуем

### Инициализация

Это должно уехать в модуль....

In [ ]:
import pq_thrift_utils as pqu

In [ ]:
import genpy.parquet.ttypes as tt

### FILE

In [ ]:
PQ_FILE = "/home/mk/mk_win/Documents/28.Arrow/data/tpcds_10g.parquet/store_sales.parquet"

In [ ]:
fLen = pqu.getFooterLen(PQ_FILE)
MD = pqu.decodeObject(PQ_FILE,-(8+fLen),tt.FileMetaData,fLen)

In [ ]:
grLens = {}
for i,gr in enumerate(MD.row_groups):
    oneColSet = set()
    for col in gr.columns:
        oneColSet.add(col.meta_data.num_values)
    if len(oneColSet)!=1:
        print("GRP:",i,"lengths:",oneColSet)
    if col.meta_data.num_values in grLens:
        grLens[col.meta_data.num_values] += 1
    else:
        grLens[col.meta_data.num_values] = 1
print(grLens)

In [ ]:
print("FILE:",PQ_FILE)
print("row number       :",MD.num_rows)
print("row groups number:",len(MD.row_groups))
print("rows per group   :", str(grLens).replace(": ","*"))

In [ ]:
ROW_GROUP = 0
COLUMN = 0

* кодировки для колонки в группе/файле
* наличие статистики по колонке (и где именно)
* наличие блум фильтров и их размеры

In [ ]:
print(MD.row_groups[ROW_GROUP].columns[COLUMN].meta_data.path_in_schema)

In [ ]:
if MD.row_groups[ROW_GROUP].columns[COLUMN].meta_data.encoding_stats:
    print(MD.row_groups[ROW_GROUP].columns[COLUMN].meta_data.encoding_stats)
else:
    print(tt.Encoding._VALUES_TO_NAMES[MD.row_groups[ROW_GROUP].columns[COLUMN].meta_data.encodings[0]])

In [ ]:
if MD.row_groups[ROW_GROUP].columns[COLUMN].meta_data.bloom_filter_offset:
    print("Bloom bitset len:",MD.row_groups[ROW_GROUP].columns[COLUMN].meta_data.bloom_filter_length)
else:
    print("(no bloom filter)")

In [ ]:
print("Min value:",MD.row_groups[0].columns[0].meta_data.statistics.min_value)

In [ ]:
import struct

In [ ]:
ba = MD.row_groups[0].columns[0].meta_data.statistics.min_value

In [ ]:
ba

In [ ]:
1785 + 

In [ ]:
122880 * 234 + 47071